# goal: perform financial analysis

## notes and questions
- the `Process` class helps to delegate the workflow to the agents - **kind of like a manager at work!** we can pass it to the `process` attribute of the `Crew` class.
    - in the example below, we pass `Process.hierarchical` to the `process` attribute.  
- the `manager_llm` attribute of the `Crew` class lets use choose the "manager" LLM to use and set its temperature.
- when using a hierarchical process, either `manager_llm` (setting an LLM to be used as the model for a manager agent crewai will create automatically) or `manager_agent` (setting an agent we define as the manager agent) is required. the manager agent coordinates the crew, delegating tasks and validating outcomes before proceeding.

## main code

In [1]:
import warnings
warnings.filterwarnings('ignore')  # don't bother showing warnings

from crewai import Agent, Task, Crew

from IPython.display import Markdown

import os
from utils import get_openai_api_key, pretty_print_result, get_serper_api_key
openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [2]:
# tools

from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [3]:
# agents

# data analyst
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


# trading strategist
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


# trade strategy advisor/optimiser
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


# risk advisor
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [4]:
# tasks

# analyse market data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)


# develop trading strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


# plan optimal trade execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


# assess trading risk
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [6]:
# assembling the crew

from crewai import Crew, Process
from langchain_openai import ChatOpenAI

financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4-turbo", temperature=0.3),
    process=Process.hierarchical,
    verbose=True
)

financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)  # this takes time to run

2025-03-03 11:16:23,489 - 8469596224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze the market data for AAPL, I need to involve the Data Analyst to handle the statistical modeling and machine learning aspects. They will need to gather, process, and analyze the data to identify trends and predict market movements. I should provide them with all the necessary details to ensure they understand the scope and objectives of the task.

Action: Delegate work to co-worker

Action Input: {
  "coworker": "Data Analyst",
  "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
  "context": "The task involves gathering real-time and historical market data for AAPL (Apple Inc.). You will need to apply statisti

In [7]:
Markdown(result)

The comprehensive risk analysis report for the trading strategies involving AAPL is as follows:

**1. Technical Analysis Strategy Risks and Mitigations:**
   - **Risks:**
     - False Signals from SMA and RSI.
     - Overreliance on Historical Data.
     - Market Volatility affecting indicator accuracy.
   - **Mitigations:**
     - Incorporate additional indicators like MACD, Bollinger Bands.
     - Use adaptive moving averages like EMA.
     - Establish realistic risk management protocols.
     - Regular review and adjustment of strategies.
     - Continuous learning and staying updated with market trends.

**2. Risk Management Strategy Risks and Mitigations:**
   - **Risks:**
     - Disadvantages of Stop-Loss Orders (Price Gaps, Market Volatility).
     - Incorrect estimation in Position Sizing.
   - **Mitigations:**
     - Use of Trailing Stops.
     - Enhanced Market Analysis for better stop-loss and position sizing.
     - Options for Hedging with put options.

**3. Scalping Strategy Risks and Mitigations:**
   - **Risks:**
     - Need for a strict exit strategy to avoid significant losses.
     - High operational demands and psychological stress.
     - Increased transaction costs due to frequent trading.
   - **Mitigations:**
     - Maintain tight stop-loss orders.
     - Ensure access to appropriate tools and real-time data.
     - Monitor liquidity and volume to support high-frequency trading.
     - Use technical indicators suitable for short time frames.

**4. Momentum Trading Strategy Risks and Mitigations:**
   - **Risks:**
     - Market Volatility impacting the effectiveness of trend following.
     - Timing risks in entering and exiting trades.
     - Accumulation of high transaction costs.
     - Psychological factors influencing trading decisions.
   - **Mitigations:**
     - Set tight stop-loss orders to manage risks.
     - Leverage additional technical indicators for better timing.
     - Diversify trading to reduce focus risks.
     - Monitor market sentiment and news closely.
     - Implement algorithmic trading tools to enhance execution timing.

Each strategy has its unique set of risks which require specific mitigation strategies to align with our firm’s risk tolerance and trading objectives. By implementing these mitigations, we can enhance the trading outcomes while managing the exposure effectively.